# Estimating CAPM

- This file can estimate CAPM for any firms that Yahoo has ticker data for.
- As written, it will estimate beta separately _for each calendar year_ in the date range you give it. But you can adjust it

- _Caveat: Yahoo Finance is more of a "quick and dirty" way to get return data. WRDS has more accurate return data._



In [1]:
# uncomment and run this to install packages
# after you successfully do that, add the comments back
#!pip install pandas_datareader 
#!pip install yfinance 

In [2]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr # you might need to install this (see above)
from datetime import datetime
import yfinance as yf

## Load asset return data 

Load your stock returns. This file uses yahoo finance.
The returns don't even have to be firms! They can be any asset. (Portfolios, mutual funds, crypto, …)

In [3]:
# choose your firms and years 
stocks = ['SBUX','AAPL','MSFT']
start  = datetime(2016, 1, 1)
end    = datetime(2016, 12, 31)

In [4]:
# download stock prices 
# here, from yahoo: not my fav source, but quick. 
# we need to do some data manipulation to get the data ready 
stock_prices         = yf.download(stocks, start , end)
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')     # reduce to just columns with this in the name
stock_prices.columns = stock_prices.columns.get_level_values(1)  # tickers as col names, works no matter order of tics
stock_prices # uncomment to print and see

[*********************100%%**********************]  3 of 3 completed


,AAPL,MSFT,SBUX
Date,,,
2016-01-04,23.977478,48.698883,49.853313
2016-01-05,23.376617,48.921055,50.187042
2016-01-06,22.919147,48.032391,49.742077
2016-01-07,21.951849,46.361694,48.509869
2016-01-08,22.067928,46.503872,48.458527
...,...,...,...
2016-12-23,27.109785,57.741341,49.513714
2016-12-27,27.281956,57.777863,49.383427
2016-12-28,27.165630,57.513077,48.940487


In [5]:
# this is wide data... so if we want to create a new variable, 
# then we have to do it once for each firm...
# what if we have 1000 firms? seems tough to do...
# make long/tidy:
stock_prices = stock_prices.stack().swaplevel().sort_index().reset_index()
stock_prices.columns = ['Firm','Date','Adj Close']
stock_prices # print - now that is formatted nicely, like CRSP! 

,Firm,Date,Adj Close
0,AAPL,2016-01-04,23.977478
1,AAPL,2016-01-05,23.376617
2,AAPL,2016-01-06,22.919147
3,AAPL,2016-01-07,21.951849
4,AAPL,2016-01-08,22.067928
...,...,...,...
751,SBUX,2016-12-23,49.513714
752,SBUX,2016-12-27,49.383427
753,SBUX,2016-12-28,48.940487
754,SBUX,2016-12-29,48.914436


In [6]:
# add return var = pct_change() function compares to prior row
# EXCEPT: don't compare for first row of one firm with last row of prior firm!
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby
stock_prices['ret'] = stock_prices.groupby('Firm')['Adj Close'].pct_change()
stock_prices['ret'] = stock_prices['ret'] *100 # convert to p.p. to match FF's convention on scaling (daily % rets)
stock_prices

,Firm,Date,Adj Close,ret
0,AAPL,2016-01-04,23.977478,NaN
1,AAPL,2016-01-05,23.376617,-2.505937
2,AAPL,2016-01-06,22.919147,-1.956955
3,AAPL,2016-01-07,21.951849,-4.220482
4,AAPL,2016-01-08,22.067928,0.528791
...,...,...,...,...
751,SBUX,2016-12-23,49.513714,-0.175090
752,SBUX,2016-12-27,49.383427,-0.263134
753,SBUX,2016-12-28,48.940487,-0.896940
754,SBUX,2016-12-29,48.914436,-0.053229


In [7]:
# this shows that my "groupby" approach works, because 
# count of ret = count of adj close MINUS # of firms 
# meaning that the first day for each firm has no return data
stock_prices.describe() 

,Date,Adj Close,ret
count,756,756.000000,753.000000
mean,2016-07-02 16:51:25.714285568,40.901149,0.041001
min,2016-01-04 00:00:00,20.798660,-7.171051
25%,2016-04-04 18:00:00,25.679221,-0.604633
50%,2016-07-03 00:00:00,46.938204,0.000000
75%,2016-09-30 18:00:00,49.887238,0.759564
max,2016-12-30 00:00:00,58.088299,6.496332
std,NaN,12.183672,1.386916


## Get the factor returns 

Above, we got the asset returns, $r_i$. 

To estimate $\alpha$ and $\beta$ in $r_i-r_f = \alpha + \beta (r_m-r_f)$, we need $(r_m-r_f)$ and $r_f$. Let's download those now.

Note: $(r_m-r_f)$ is the excess return on the market, which is _one_ "factor". Modern asset pricing typically uses 5 factors in tests.


In [8]:
# We need (r_mkt - rf), and rf
# the Fama French data library is a benchmark asset pricing dataset 
ff = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',start=start,end=end)[0] # the [0] is because the imported obect is a dictionary, and key=0 is the dataframe
ff = ff.reset_index().rename(columns={"Mkt-RF":"mkt_excess"})
ff

C:\Users\DonsLaptop\AppData\Local\Temp\ipykernel_22820\2497046008.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily',start=start,end=end)[0] # the [0] is because the imported obect is a dictionary, and key=0 is the dataframe


,Date,mkt_excess,SMB,HML,RMW,CMA,RF
0,2016-01-04,-1.59,-0.76,0.52,0.35,0.42,0.000
1,2016-01-05,0.12,-0.24,0.01,0.05,0.31,0.000
2,2016-01-06,-1.35,-0.23,0.00,0.15,0.04,0.000
3,2016-01-07,-2.44,-0.29,0.08,0.49,0.36,0.000
4,2016-01-08,-1.11,-0.52,-0.03,0.25,0.05,0.000
...,...,...,...,...,...,...,...
247,2016-12-23,0.19,0.53,-0.50,-0.36,-0.15,0.001
248,2016-12-27,0.27,0.23,0.14,0.17,0.04,0.001
249,2016-12-28,-0.87,-0.29,0.09,0.17,-0.15,0.001
250,2016-12-29,-0.04,0.10,-0.33,0.27,0.02,0.001


## Merge the asset and factor returns 

In [9]:
assets_and_factors = pd.merge(
    left=stock_prices,
    right=ff,
    on="Date",
    how="inner",
    indicator=True,
    validate="many_to_one",
)
assets_and_factors

,Firm,Date,Adj Close,ret,mkt_excess,SMB,HML,RMW,CMA,RF,_merge
0,AAPL,2016-01-04,23.977478,NaN,-1.59,-0.76,0.52,0.35,0.42,0.000,both
1,MSFT,2016-01-04,48.698883,NaN,-1.59,-0.76,0.52,0.35,0.42,0.000,both
2,SBUX,2016-01-04,49.853313,NaN,-1.59,-0.76,0.52,0.35,0.42,0.000,both
3,AAPL,2016-01-05,23.376617,-2.505937,0.12,-0.24,0.01,0.05,0.31,0.000,both
4,MSFT,2016-01-05,48.921055,0.456215,0.12,-0.24,0.01,0.05,0.31,0.000,both
...,...,...,...,...,...,...,...,...,...,...,...
751,MSFT,2016-12-29,57.430901,-0.142883,-0.04,0.10,-0.33,0.27,0.02,0.001,both
752,SBUX,2016-12-29,48.914436,-0.053229,-0.04,0.10,-0.33,0.27,0.02,0.001,both
753,AAPL,2016-12-30,26.946926,-0.779594,-0.52,-0.06,0.20,-0.11,0.03,0.001,both
754,MSFT,2016-12-30,56.736992,-1.208250,-0.52,-0.06,0.20,-0.11,0.03,0.001,both


## Estimate CAPM

So the data’s basically ready. _(We need to do two quick things below.)_

Again, the goal is to estimate, **for each** firm, **for each** year, alpha and beta, from the CAPM formula.

Well, as we've said, if you are doing a "for each" on a dataframe, that means you want to use groupby!

So, I have a dataframe, and **for each** firm, and **for each** year, I want to \<do stuff> (run regressions).

That almost directly translates to the code we need: `assets_and_factors.groupby([firm,year]).runregression()`. Except there is no "runregression" function that applies to pandas groupby objects. But we can write such a function and then `apply()` it. Meaning, our plan is to basically use this code: `assets_and_factors.groupby([firm,year]).apply(<our own reg fcn>)`.

We just need to write a reg function that works on groupby objects.

In [10]:
import statsmodels.api as sm


def reg_in_groupby(df, formula="ret_excess ~ mkt_excess + SMB + HML"):
    """
    Want to run regressions after groupby? E.g., repeat the regression 
    for each firm-year?
    
    This will do it! 
    
    Note: This defaults to a FF3 model assuming specific variable names. If you
    want to run any other regression, just specify your model.
    
    Usage: 
        df.groupby(<whatever>).apply(reg_in_groupby)
        df.groupby(<whatever>).apply(reg_in_groupby,formula=<whatever>)
    """
    return pd.Series(sm.formula.ols(formula, data=df).fit().params)

In [11]:
(
    assets_and_factors # grab the data
    
    # Two things before the regressions:
    # 1. need a year variable (to group on)
    # 2. the market returns in FF are excess returns, so 
    #    our stock returns need to be excess as well
    .assign(year = assets_and_factors.Date.dt.year,
            ret_excess = assets_and_factors.ret - assets_and_factors.RF)
    
    # ok, run the regs, so easy!
    .groupby(['Firm','year']).apply(reg_in_groupby,formula='ret_excess ~ mkt_excess')
    
    # and clean up - with better var names
    .rename(columns={'Intercept':'alpha','mkt_excess':'beta'})
    .reset_index()
)

,Firm,year,alpha,beta
0,AAPL,2016,-0.001152,0.965515
1,MSFT,2016,0.000603,1.165980
2,SBUX,2016,-0.056930,0.848169
